In [ ]:
##предварительная обработка текста
import numpy as np
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords

##Чтение датасета с текстом
df = pd.read_excel(r"C:\Users\Kirill\OneDrive\Desktop\study\Технологии автоматической обработки текста\myreviews.xlsx")
df['text_lower']  = df['Текст рецензии'].str.lower()
df['text_punct'] = df['text_lower'].str.replace('[^\w\s]',' ', regex=True)
STOPWORDS = set(stopwords.words('russian'))
# функция, удаляющая стопслова из текстов

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
# Применим функцию удаления стоп-слов
df["text_stop"] = df["text_punct"].apply(stopwords)
df['text_nonum'] = df['text_stop'].str.replace('\d+', '')
#Можно выполнить при помощи NLTK. Заодно проведем лемматизацию
from pymorphy3 import MorphAnalyzer
from nltk import sent_tokenize, word_tokenize, regexp_tokenize

def tokenize_lemmas(sent, pat=r"(?u)\b\w\w+\b", morph=MorphAnalyzer()):
    return [morph.parse(tok)[0].normal_form 
            for tok in regexp_tokenize(sent, pat)]
df["text_lemm"] = df["text_nonum"].map(lambda x: " ".join(tokenize_lemmas(x)))


<>:12: SyntaxWarning: invalid escape sequence '\w'
<>:20: SyntaxWarning: invalid escape sequence '\d'
<>:12: SyntaxWarning: invalid escape sequence '\w'
<>:20: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Kirill\AppData\Local\Temp\ipykernel_2104\3515941204.py:12: SyntaxWarning: invalid escape sequence '\w'
  df['text_punct'] = df['text_lower'].str.replace('[^\w\s]',' ', regex=True)
C:\Users\Kirill\AppData\Local\Temp\ipykernel_2104\3515941204.py:20: SyntaxWarning: invalid escape sequence '\d'
  df['text_nonum'] = df['text_stop'].str.replace('\d+', '')


In [19]:
# Преобразование целевой переменной
df['Target'] = df['Общая оценка'].apply(lambda x: 1 if x >= 70 else 0)

# Выбор признаков
X_text = df['text_lemm']
y = df['Target']

from sklearn.feature_extraction.text import TfidfVectorizer

# Векторизация текста
vectorizer = TfidfVectorizer(max_features=500)  # Ограничим количество признаков до 500
X_text_transformed = vectorizer.fit_transform(X_text)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_text_transformed, y, test_size=0.2, random_state=42
)

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Обучение модели
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Предсказания
y_pred_nb = nb_model.predict(X_test)

# Метрики
print("Байесовский классификатор:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_nb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_nb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_nb):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_nb):.4f}")

Байесовский классификатор:
Accuracy: 0.7522
Precision: 0.7475
Recall: 0.9931
F1-score: 0.8530


In [21]:
from sklearn.tree import DecisionTreeClassifier

# Обучение модели
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

# Предсказания
y_pred_tree = tree_model.predict(X_test)

# Метрики
print("Дерево решений:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_tree):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_tree):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_tree):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_tree):.4f}")


Дерево решений:
Accuracy: 0.7019
Precision: 0.7940
Recall: 0.7942
F1-score: 0.7941


In [22]:
from sklearn.ensemble import RandomForestClassifier

# Обучение модели
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Предсказания
y_pred_rf = rf_model.predict(X_test)

# Метрики
print("Случайный лес:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_rf):.4f}")


Случайный лес:
Accuracy: 0.7856
Precision: 0.7948
Recall: 0.9487
F1-score: 0.8650


In [ ]:
from sklearn.linear_model import LogisticRegression

# Обучение модели
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)

# Предсказания
y_pred_lr = lr_model.predict(X_test)

# Метрики
print("Логистическая регрессия:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_lr):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_lr):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_lr):.4f}")


Логистическая регрессия:
Accuracy: 0.8046
Precision: 0.8273
Recall: 0.9226
F1-score: 0.8724


In [24]:
from sklearn.svm import LinearSVC

# Обучение модели
svc_model = LinearSVC(random_state=42)
svc_model.fit(X_train, y_train)

# Предсказания
y_pred_svc = svc_model.predict(X_test)

# Метрики
print("Метод опорных векторов (SVM):")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svc):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_svc):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_svc):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_svc):.4f}")

Метод опорных векторов (SVM):
Accuracy: 0.8044
Precision: 0.8251
Recall: 0.9262
F1-score: 0.8727
